<a href="https://colab.research.google.com/github/afujii/Hyakunin/blob/main/GeneticAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

オライリー出版の「集合知プログラミング」第11章の「遺伝的プログラミング」を以下に示す。遺伝アルゴルリズムを処理プログラムの構造に反映させたものといえる。

In [ ]:
from random import random,randint,choice
from copy import deepcopy
from math import log

プログラムは木構造で構成され、各ノードが処理単位となる。関数は名前で管理される。Pythonでは、関数名を渡すことで処理が可能である。以下で、lambda 関数によって処理を定義し、その名前の関数をノードに置いた木を構成することで処理全体を「木」で表す。その出力と目的関数の誤差を最小化するように、枝を「変異」「交叉」させる。

In [ ]:
class node:
  def __init__(self,fw,children):
    self.function=fw.function
    self.name=fw.name
    self.children=children

  def evaluate(self,inp):
    results=[n.evaluate(inp) for n in self.children]
    return self.function(results)

  def display(self,indent=1):
    print ((' '*indent)+self.name)
    for c in self.children:
      c.display(indent+1)

class paramnode:
  def __init__(self,idx):
    self.idx=idx
  def evaluate(self,inp):
    return inp[self.idx]
  def display(self,indent=0):
    print( ' '*indent + str(self.idx))

class constnode:
  def __init__(self,v):
    self.v=v
  def evaluate(self,inp):
    return self.v
  def display(self,indent=0):
    print( ' '*indent + str(self.v ) )


ノードの「display」は、インデントによって木の深さを表現して表示する機能を定義している。

まず、例題として、加減乗の演算を行う。乱数により任意の構造の木を生成する機能を定義する。以下のfwrapper によって、関数のラッパー(wrapper)を定義する。また、大商比較の関数、ＩF条件文を表す関数を用意する。

In [ ]:
class fwrapper:
  def __init__(self,function,childcount,name):
    self.function=function
    self.childcount=childcount
    self.name=name

In [ ]:
addw=fwrapper(lambda l:l[0]+l[1],2,'add')
subw=fwrapper(lambda l:l[0]-l[1],2,'subtract')
mulw=fwrapper(lambda l:l[0]*l[1],2,'multiply')

def iffunc(l):
  if l[0]>0: return l[1]
  else: return l[2]
ifw=fwrapper(iffunc,3,'if')

def isgreater(l):
  if l[0]>l[1]: return 1
  else: return 0

In [ ]:
gtw=fwrapper(isgreater,2,'isgreater')
flist=[addw,mulw,ifw,gtw,subw]

In [ ]:
def exampletree():
  return node(ifw,[
                  node(gtw,[paramnode(0),constnode(3)]),
                  node(addw,[paramnode(1),constnode(5)]),
                  node(subw,[paramnode(1),constnode(2)]),
                  ]
              )

例題として生成された関数の構造を見てみよう。

In [ ]:
sample = exampletree()
sample.display()

 if
  isgreater
   0
   3
  add
   1
   5
  subtract
   1
   2


次に、突然変異、交叉、を導入して関数に対して遺伝アルゴリズムの処理を適用させるための準備を行う。

In [ ]:
def makerandomtree(pc,maxdepth=4,fpr=0.5,ppr=0.6):
  if random()<fpr and maxdepth>0:
    f=choice(flist)
    children=[makerandomtree(pc,maxdepth-1,fpr,ppr)
              for i in range(f.childcount)]
    return node(f,children)
  elif random()<ppr:
    return paramnode(randint(0,pc-1))
  else:
    return constnode(randint(0,10))

def mutate(t,pc,probchange=0.1):
  if random()<probchange:
    return makerandomtree(pc)
  else:
    result=deepcopy(t)
    if hasattr(t,"children"):
      result.children=[mutate(c,pc,probchange) for c in t.children]
    return result

def crossover(t1,t2,probswap=0.7,top=1):
  if random()<probswap and not top:
    return deepcopy(t2)
  else:
    result=deepcopy(t1)
    if hasattr(t1,'children') and hasattr(t2,'children'):
      result.children=[crossover(c,choice(t2.children),probswap,0)
                       for c in t1.children]
    return result

def getrankfunction(dataset):
  def rankfunction(population):
    scores=[(scorefunction(t,dataset),t) for t in population]
    scores = sorted(scores, key = lambda x: x[0])
    # オリジナルのコードでは、sort()　を使っており、
    # python3 系では、関数渡しでは、scores の中身が変化しない。
    # 上記のような　scores　の再定義が必要であった。
    return scores
  return rankfunction

makerandomtree で初期の木を作る。
世代の進化を行う関数を導入する。生成された関数を名前渡しし、その関数の木構造を変形することによって進化させる。

In [ ]:
def evolve(pc,popsize,rankfunction,maxgen=500,
           mutationrate=0.1,breedingrate=0.4,pexp=0.7,pnew=0.05):
  # Returns a random number, tending towards lower numbers. The lower pexp
  # is, more lower numbers you will get
  def selectindex():
    return int(log(random())/log(pexp))

  # Create a random initial population
  population=[makerandomtree(pc) for i in range(popsize)]
  for i in range(maxgen):
    scores=rankfunction(population)
    print( scores[0][0] )
    if scores[0][0]==0: break

    # The two best always make it
    newpop=[scores[0][1],scores[1][1]]

    # Build the next generation
    while len(newpop)<popsize:
      if random()>pnew:
        newpop.append(mutate(
                      crossover(scores[selectindex()][1],
                                 scores[selectindex()][1],
                                probswap=breedingrate),
                        pc,probchange=mutationrate))
      else:
      # Add a random node to mix things up
        newpop.append(makerandomtree(pc))

    population=newpop
  scores[0][1].display()
  return scores[0][1]


「進化」を評価するには、評価関数が必要である。ここでは、例題として隠された正解の関数が、ここでは、例題として隠された正解の関数が、$x^2+2y+3x+5$とする。また、精製されたプログラムをテストするデータセットを生成する関数を用意する。
buildhiddenset により、任意のx,y の組に　正解の値を対応させたものを　200個生成する。これを利用して、プログラムの途中で生成される木の評価を行う。

In [ ]:
def hiddenfunction(x,y):
    return x**2+2*y+3*x+5

def buildhiddenset():
  rows=[]
  for i in range(200):
    x=randint(0,40)
    y=randint(0,40)
    rows.append([x,y,hiddenfunction(x,y)])
  return rows

def scorefunction(tree,s):
  dif=0
  for data in s:
    v=tree.evaluate([data[0],data[1]])
    dif+=abs(v-data[2])
  return dif

任意に生成した木（関数 sample）の構造と評価値を確認しよう。

In [ ]:
h = buildhiddenset()
# print(h)
scorefunction(sample,h)

115520

では、進化させてその途中の評価と結果を確認しよう。評価が高い、つまり真の値との差が小さいものから順に並べた　scores リストの上位から遺伝子を取り出して操作し、世代交代を行う。うまく収束する場合と、ほとんど変化なく進展する場合がある。

In [ ]:
rf = getrankfunction(buildhiddenset())
evolve(2,500,rf,mutationrate=0.2,breedingrate=0.1,pexp=0.7,pnew=0.1)

9458
6090
4664
2940
200
200
200
199
199
199
199
199
199
199
199
199
199
199
199
199
1
0
 add
  multiply
   0
   add
    0
    3
  add
   5
   add
    1
    if
     1
     1
     isgreater
      1
      1


結果の関数の構造は複雑に見えるが、与えられた式と同等の結果を出す関数であることを確かめられたい。以下では、この関数の生成過程に基づいて学習したプログラムによって簡単な2人ゼロ和ゲーム（○×）をプレイする例を示す。

In [ ]:
def gridgame(p):
  # Board size
  max=(3,3)

  # Remember the last move for each player
  lastmove=[-1,-1]

  # Remember the player's locations
  location=[[randint(0,max[0]),randint(0,max[1])]]

  # Put the second player a sufficient distance from the first
  location.append([(location[0][0]+2)%4,(location[0][1]+2)%4])
  # Maximum of 50 moves before a tie
  for o in range(50):

    # For each player
    for i in range(2):
      locs=location[i][:]+location[1-i][:]
      locs.append(lastmove[i])
      move=p[i].evaluate(locs)%4

      # You lose if you move the same direction twice in a row
      if lastmove[i]==move: return 1-i
      lastmove[i]=move
      if move==0:
        location[i][0]-=1
        # Board wraps
        if location[i][0]<0: location[i][0]=0
      if move==1:
        location[i][0]+=1
        if location[i][0]>max[0]: location[i][0]=max[0]
      if move==2:
        location[i][1]-=1
        if location[i][1]<0: location[i][1]=0
      if move==3:
        location[i][1]+=1
        if location[i][1]>max[1]: location[i][1]=max[1]

      # If you have captured the other player, you win
      if location[i]==location[1-i]:
        #print('player'+str(i)+'win')
        return i
  return -1

In [ ]:
def tournament(pl):
  # Count losses
  losses=[0 for p in pl]

  # Every player plays every other player
  for i in range(len(pl)):
    for j in range(len(pl)):
      if i==j: continue

      # Who is the winner?
      winner=gridgame([pl[i],pl[j]])

      # Two points for a loss, one point for a tie
      if winner==0:
        losses[j]+=2
      elif winner==1:
        losses[i]+=2
      elif winner==-1:
        losses[i]+=1
        losses[i]+=1
  # Sort and return the results
  #import pdb; pdb.set_trace()
  #z = zip( losses,pl)
  #z.sort() # corrected by A.Fujii 2018/07/01
  z=sorted( zip(losses,pl), key=lambda x: x[0] )
  return z

In [ ]:
class humanplayer:
  def evaluate(self,board):

    # Get my location and the location of other players
    me=tuple(board[0:2])
    others=[tuple(board[x:x+2]) for x in range(2,len(board)-1,2)]

    # Display the board
    for i in range(4):
      for j in range(4):
        if (i,j)==me:
          print ('O', end='')
        elif (i,j) in others:
          print ('X', end='')
        else:
          print ('.', end='')
      print()

    # Show moves, for reference
    print ('Your last move was %d' % board[len(board)-1])
    print (' 0')
    print ('2 3')
    print (' 1')
    print ('Enter move: ',end='')

    # Return whatever the user enters
    #move=int(raw_input())
    move = int(input()) #A.Fujii for 3.4 2018/07/01
    return move

In [ ]:
p1 = makerandomtree(5)
p2 = makerandomtree(5)
gridgame([p1,p2])

1

In [ ]:
winner = evolve(5,100,tournament,maxgen=50)

6
60
78
96
88
78
68
70
90
86
78
64
66
84
60
104
68
80
80
78
68
80
78
76
58
84
86
70
84
78
84
94
72
84
88
76
86
80
76
64
56
58
56
40
52
60
62
64
56
52
 subtract
  4
  if
   multiply
    isgreater
     subtract
      add
       2
       isgreater
        2
        0
      isgreater
       multiply
        if
         4
         3
         subtract
          4
          if
           subtract
            2
            if
             3
             4
             add
              9
              add
               9
               multiply
                4
                1
           add
            if
             add
              3
              9
             1
             subtract
              0
              4
            add
             if
              subtract
               2
               4
              2
              4
             isgreater
              5
              7
           3
        if
         isgreater
          if
           isgreater
            5
     

学習済みのコンピュータ指しては、X側である。学習済み関数は、O側から逃れるの様に学習したとみなす。（入力数値に対する動作を出力する）
ルールとしては、「同じ動きを連続して行うと負け(0)」「相手を壁に追い詰めて補足すると勝ち(1)」である。動ける場所は、4×4の16マスである。

In [ ]:
gridgame([winner,humanplayer( )])

....
O...
....
..X.
Your last move was -1
 0
2 3
 1
Enter move: 1
....
....
O.X.
....
Your last move was 1
 0
2 3
 1
Enter move: 3


0